# Cell 1 - import vari

In [6]:
import os
import joblib
import torch
import torch.nn as nn
import pandas as pd

In [7]:
MODEL_DIR = os.path.join("..", "models")
scaler_path = os.path.join(MODEL_DIR, "scaler_audio_features.pkl")
model_path = os.path.join(MODEL_DIR, "music_mood_mlp.pt")

In [8]:
class MusicMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# Cell 2 - Setting and Running the model

In [9]:
scaler = joblib.load(scaler_path)
print("Scaler caricato.")

Scaler caricato.


In [10]:
feature_cols = [
    "acousticness", "danceability", "energy", "instrumentalness",
    "liveness", "loudness", "speechiness", "tempo", "valence",
    "duration_ms"
]

num_classes = 8  

input_dim = len(feature_cols)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MusicMLP(input_dim=input_dim, num_classes=num_classes).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

print("Modello caricato su:", device)

Modello caricato su: cpu


In [11]:
DATA_PROCESSED_DIR = os.path.join("..", "data", "processed")
data_path = os.path.join(DATA_PROCESSED_DIR, "spotify_dataset_clustered.csv")

df = pd.read_csv(data_path)
print("Dataset loaded:", df.shape)
df.head()

Dataset loaded: (169909, 18)


,track_id,track_name,artist_name,popularity,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,duration_ms,pca_x,pca_y,cluster
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],0,1928,0.995,0.708,0.1950,0.563,0.1510,-12.428,0.0506,118.469,0.7790,158648,-1.010631,1.593194,7
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",0,1928,0.994,0.379,0.0135,0.901,0.0763,-28.454,0.0462,83.972,0.0767,282133,-4.751081,-0.113671,0
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],0,1928,0.604,0.749,0.2200,0.000,0.1190,-19.924,0.9290,107.177,0.8800,104300,-0.184709,4.573615,7
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],0,1928,0.995,0.781,0.1300,0.887,0.1110,-14.734,0.0926,108.003,0.7200,180760,-1.671672,1.857104,5
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",1,1928,0.990,0.210,0.2040,0.908,0.0980,-16.829,0.0424,62.149,0.0693,687733,-4.018761,-2.630070,0


In [12]:

df_model = df.dropna(subset=feature_cols).copy()
print("Dataset after dropna:", df_model.shape)

Dataset after dropna: (169909, 18)


In [13]:
X = df_model[feature_cols].values.astype("float32")

X_scaled = scaler.transform(X)

with torch.no_grad():
    inputs = torch.tensor(X_scaled, dtype=torch.float32).to(device)
    outputs = model(inputs)
    predicted_classes = torch.argmax(outputs, dim=1).cpu().numpy()

df_model["cluster"] = predicted_classes

print("Clusters assigned to dataset.")

Clusters assigned to dataset.


# Cell 4 - engnieering the output 

In [14]:
import numpy as np

# Drop PCA columns if they exist
df_model = df_model.drop(columns=["pca_x", "pca_y"])

# Compute distances to centroids
df_model['distance_to_centroid'] = 0.0

for cluster_id in df_model["cluster"].unique():
    
    cluster_data = df_model[df_model["cluster"] == cluster_id]
    cluster_features = cluster_data[feature_cols].values
    centroid = cluster_features.mean(axis=0)
    distances = np.linalg.norm(cluster_features - centroid, axis=1)
    df_model.loc[df_model["cluster"] == cluster_id, 'distance_to_centroid'] = distances

# Compute normalized distance within each cluster if we need it
df_model['distance_normalized'] = df_model.groupby('cluster')['distance_to_centroid'] \
                                           .transform(lambda x: (x - x.min()) / (x.max() - x.min()))

# computing metrics for ranking
df_model['rank_distance'] = df_model.groupby('cluster')['distance_to_centroid'] \
                                    .rank(ascending=True)  

df_model['rank_popularity'] = df_model.groupby('cluster')['popularity'] \
                                 .rank(ascending=False)

weight_distance = 0.4
weight_pop = 0.6
df_model['combined_rank'] = weight_distance * (1 - df_model['rank_distance'] / df_model.groupby('cluster')['rank_distance'].transform('max')) + \
                            weight_pop * (1 - df_model['rank_popularity'] / df_model.groupby('cluster')['rank_popularity'].transform('max'))

# final output
df_model.head()

,track_id,track_name,artist_name,popularity,year,acousticness,danceability,energy,instrumentalness,liveness,...,speechiness,tempo,valence,duration_ms,cluster,distance_to_centroid,distance_normalized,rank_distance,rank_popularity,combined_rank
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],0,1928,0.995,0.708,0.1950,0.563,0.1510,...,0.0506,118.469,0.7790,158648,7,685.398548,0.001083,147.0,9342.5,0.395381
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",0,1928,0.994,0.379,0.0135,0.901,0.0763,...,0.0462,83.972,0.0767,282133,0,29682.510715,0.005819,1413.5,11276.0,0.359136
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],0,1928,0.604,0.749,0.2200,0.000,0.1190,...,0.9290,107.177,0.8800,104300,7,55033.330516,0.087828,9180.0,9342.5,0.111525
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],0,1928,0.995,0.781,0.1300,0.887,0.1110,...,0.0926,108.003,0.7200,180760,5,4931.601819,0.004455,2350.0,16483.5,0.353729
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",1,1928,0.990,0.210,0.2040,0.908,0.0980,...,0.0424,62.149,0.0693,687733,0,375917.494190,0.073820,13042.0,8472.0,0.172156


# Cell 4 - Playlist creation function

In [ ]:
from collections import Counter
from datetime import datetime
def recommend_playlist(mood, activity, time_of_day, age):

    #moody matrix
    mood_map = {
        "relax": [0,2,4,5],
        "happy": [1,4,7],
        "sad": [2],
        "workout": [3,6],
        "focus": [0,5],
        "party": [1,7]
    }

    activity_map = {
        "study time": [0,5],
        "walking": [1,4,7],
        "running": [6,3],
        "relaxing": [0,2],
        "party": [1,7]
    }

    time_map = {
        "morning": [7,4],
        "afternoon": [1,4],
        "evening": [0,2,5],
        "night": [0,5]
    }

    #cluster selection

    clusters = []
    clusters += mood_map.get(mood, [])
    clusters += activity_map.get(activity, [])
    clusters += time_map.get(time_of_day, [])

    if mood == "party" and activity == "party" and time_of_day == 'night':
        clusters.remove(0)
        clusters.remove(5)
        clusters += [1,7] 

    # first 10/15 songs from weighted clusters based popularity and distance to centroid
    Coun = dict(Counter(clusters))
    cluster_sum = len(clusters)
    if mood in ("party", "happy"):
        playlist_length = 10
    else:
        playlist_length = 15

    weighted_clusters = {}

    for cluster, count in Coun.items():
        weight = count / cluster_sum
        n_songs = max(1, int(weight * playlist_length))
        weighted_clusters[cluster] = n_songs

    candidated_songs = pd.DataFrame()
    
    for cluster, count in weighted_clusters.items():
        candidated_songs = pd.concat([
        candidated_songs,
        df_model[df_model["cluster"] == cluster].sort_values('combined_rank', ascending = False).head(count)
    ])


    # age filtering for party and happy mood
    year = datetime.now().year
    birth_year = int(year) - int(age)
    
    if mood in ("party", "happy"):
        candidated_songs = pd.concat([
        candidated_songs,
        df_model[(df_model["cluster"] == max(Coun, key=Coun.get)) & ((df_model["year"] >= birth_year + 18) & (df_model["year"] <= birth_year + 35)) ].sort_values('popularity', ascending = False).head(5)
    ])
    
    
    return candidated_songs.head(20)


recommend_playlist('party','party','night','24')


,track_id,track_name,artist_name,popularity,year,acousticness,danceability,energy,instrumentalness,liveness,...,speechiness,tempo,valence,duration_ms,cluster,distance_to_centroid,distance_normalized,rank_distance,rank_popularity,combined_rank
87959,11VApNQCWLJdzxWrlmwzUa,Say So (feat. Nicki Minaj),"['Doja Cat', 'Nicki Minaj']",89,2020,0.11900,0.856,0.641,0.000005,0.4270,...,0.2040,111.004,0.745,206221,1,105.288193,0.000238,43.0,22.5,0.998862
116670,2lCkncy6bIB0LTMT7kvrD1,Azul,['J Balvin'],86,2020,0.08160,0.843,0.836,0.001380,0.0532,...,0.0695,94.018,0.650,205933,1,184.523098,0.000425,77.0,47.0,0.997811
87981,7wsmIIm0xWmtP7TmACXkJn,Oprah’s Bank Account (Lil Yachty & DaBaby feat...,"['Lil Yachty', 'DaBaby', 'Drake']",83,2020,0.17000,0.836,0.617,0.000000,0.3190,...,0.0534,138.066,0.866,206702,1,586.422392,0.001378,224.0,93.0,0.994612
87552,0KKkJNfGyhkQ5aFogxQAPU,That's What I Like,['Bruno Mars'],81,2016,0.01300,0.853,0.560,0.000000,0.0944,...,0.0406,134.066,0.860,206693,1,577.293625,0.001357,218.0,151.5,0.993389
87581,6g0Orsxv6glTJCt4cHsRsQ,Formation,['Beyoncé'],77,2016,0.00532,0.896,0.621,0.000000,0.1960,...,0.2370,121.966,0.818,206080,1,36.320961,0.000074,13.0,383.0,0.991227
144052,6tAqYm2Wcy2yrPixShJMS6,Alone,"['SadBoyProlific', 'Ivri']",69,2018,0.48100,0.870,0.341,0.002700,0.3290,...,0.4390,119.054,0.436,160105,7,771.733295,0.001221,169.0,49.5,0.991510
144303,2HnfqILzgKt1CepTlU4oqq,Like No One Does,['Jake Scott'],69,2020,0.35200,0.800,0.410,0.000001,0.1070,...,0.2360,104.941,0.442,158469,7,864.344419,0.001369,189.0,49.5,0.990882
86594,6RFkVsPmrM4pzlDkFswwJl,God's Gonna Cut You Down,['Johnny Cash'],64,2006,0.86800,0.617,0.485,0.000034,0.1150,...,0.1150,82.228,0.845,158573,7,760.840727,0.001204,166.0,108.0,0.987848
96040,3gqgSM82j6NMAj4Jllr06T,Vete Ya,['Valentín Elizalde'],69,2003,0.50400,0.811,0.323,0.000016,0.0190,...,0.0744,88.018,0.764,157760,7,1573.473614,0.002501,332.0,49.5,0.986388
95162,373yGkNo74RZgmQgTiR8xK,It's Only A Paper Moon,"['Ella Fitzgerald', 'The Delta Rhythm Boys']",61,1994,0.94400,0.761,0.103,0.000000,0.1270,...,0.0416,114.044,0.530,160133,7,799.686490,0.001266,175.0,171.5,0.983487
